In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("huyydangg/DEk21_hcmute_embedding")

sentences = [
    "Điều 2 Quyết định 185/QĐ-UB năm 1998 Bảng giá đất tỉnh Bến Tre có nội dung như sau:\n\nĐiều 2. Giá đất trên được áp dụng cho những trường hợp: Tính thuế chuyển quyền sử dụng cho những trường hợp: Tính thuế chuyển quyền sử dụng đất, thu lệ phí trước bạ, thu tiền sử dụng đất khi giao đất, cho thuê đất, tính giá trị tài sản khi giao đất, bồi thường thiệt hại về đất khi Nhà nước thu hồi.\nTrường hợp giao đất theo hình thức đấu giá, thì giá đất sẽ do Uỷ ban nhân dân tỉnh cho trường hợp cụ thể.\nGiá cho thuê đất đối với các tổ chức, cá nhân nước ngoài hoặc xí nghiệp có vốn đầu tư nước ngoài được áp dụng theo quy định của Chính phủ.",
    "Điều 2 Quyết định 55/2012/QĐ-UBND dự toán ngân sách phân bổ dự toán ngân sách 2013 Bình Dương",
    "Điều 2 Quyết định 185/QĐ-UB năm 1998 Bảng giá đất tỉnh Bến Tre",
    "Điều 3 Quyết định 79/2019/QĐ-UBND mức thu học phí quản lý và sử dụng học phí giáo dục mầm non Huế"
]
embeddings = model.encode(sentences)

similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)
# [4, 4]

/Users/btk0286/hailt/multi_agents/notebook/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/btk0286/hailt/multi_agents/notebook/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


torch.Size([4, 4])


In [7]:
embeddings = model.encode(
    sentences,
    show_progress_bar=True,
    chunk_size=1024,
    batch_size=1024
    )
embeddings.shape

Batches:   0%|          | 0/1 [00:00<?, ?it/s]/Users/btk0286/hailt/multi_agents/notebook/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Batches: 100%|██████████| 1/1 [00:00<00:00,  2.54it/s]


(4, 768)

In [12]:
from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer
import uuid
import os

# 1. Khởi tạo Qdrant client (localhost hoặc remote)
# client = QdrantClient(host="localhost", port=6333)
client = QdrantClient(
    url=os.getenv("QDRANT_URL"),
    api_key=os.getenv('QDRANT_CLOUD_API_KEY')
)
# 2. Load mô hình embedding từ HuggingFace
model =SentenceTransformer("huyydangg/DEk21_hcmute_embedding")


# 3. Tạo collection nếu chưa có
collection_name = os.getenv("QDRANT_COLLECTION")

if not client.collection_exists(collection_name=collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config={
            "dense_vector": models.VectorParams(
                size=768,
                distance=models.Distance.COSINE,
            )
        },
        sparse_vectors_config={
            "bm25_sparse_vector": models.SparseVectorParams(
                modifier=models.Modifier.IDF # Enable BM25
            )
        }
    )

# 4. Danh sách văn bản cần nhúng
texts = [
    "Xin chào, bạn cần gì?",
    "Tôi đang tìm kiếm nhà hàng gần đây",
    "Thời tiết hôm nay thế nào?"
]

# 5. Tạo vector embedding
dense_vectors = model.encode(texts).tolist()

# 6. Chuyển thành payload và insert vào Qdrant
points = [
    models.PointStruct(
        id=str(uuid.uuid4()),  # ID tự sinh, hoặc dùng int
        vector={"dense_vector": dense_vectors[i]},
        payload={"text": texts[i]}
    )
    for i in range(len(texts))
]

client.upsert(collection_name=collection_name, points=points)
print("✅ Đã insert vào Qdrant!")


✅ Đã insert vào Qdrant!


In [14]:
from qdrant_client import models

collection_name = "my_collection_name"

if not client.collection_exists(collection_name=collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config={
            "dense_vector": models.VectorParams(
                size=1024,
                distance=models.Distance.COSINE
            )
        },
        sparse_vectors_config={
            "bm25_sparse_vector": models.SparseVectorParams(
                modifier=models.Modifier.IDF # Enable Inverse Document Frequency
            )
        }
    )